## Segmenting Neighborhoods in Toronto Notebook

In [93]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
! wget -O postalcodesT https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


--2020-04-10 14:18:47--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org... 2620:0:863:ed1a::1, 198.35.26.96
Connecting to en.wikipedia.org|2620:0:863:ed1a::1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52229 (51K) [text/html]
Saving to: 'postalcodesT'

postalcodesT        100%[===================>]  51.00K  --.-KB/s    in 0.04s   

2020-04-10 14:18:47 (1.14 MB/s) - 'postalcodesT' saved [52229/52229]



In [16]:
# read wget file. I got lucky as I saw it was a list and put[0] after and found that it read it all. 
df = pd.read_html('postalcodesT')[0]
df

Postal code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                          Neighborhood  
0                                                  NaN  
1                                                  NaN  
2                                            Parkwoods  
3                                     Victoria Village  
4                           Regent Park / Harbourfront  
..                                                 ...  
175                                                NaN  
176                                                NaN  
177                                                NaN  
178  Mimico NW / The Queensway West / South of Bloo...  
179                                                NaN  

[180 rows x 3 columns]

In [20]:
# find all the Boroughs who do not have a name and drop them
not_assigned_index = df.index[df['Borough'] == 'Not assigned']
df.drop(index=not_assigned_index, inplace=True)  

In [30]:
# There are no Neighborhoods who have Null values
df[df['Neighborhood'].isnull()]

Empty DataFrame
Columns: [Postal code, Borough, Neighborhood]
Index: []

In [33]:
#Reset Dataframe Index for clarity's sake
df = df.reset_index(drop=True)

Postal code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
..          ...               ...   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                           Regent Park / Harbourfront  
3                    Lawrence Manor / Lawrence Heights  
4         Queen's Park / Ontario Provincial Government  
..                                                 ...  
98     The Kingsway / Montgomery Road / Old Mill North  
99                                Church and Wellesley  
100              Business reply mail Processing CentrE  
101  Old Mill South / King's Mill Park / Sunnylea /...  
102  Mimico NW / The Queensway West / South of Bloo...  

[103 rows x 3 columns]

In [34]:
df.shape

(103, 3)

# End Part One

# PART TWO

In [38]:
!wget http://cocl.us/Geospatial_data

--2020-04-10 14:36:11--  http://cocl.us/Geospatial_data
Resolving cocl.us... 169.48.113.194, 158.85.108.83, 158.85.108.86
Connecting to cocl.us|169.48.113.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-04-10 14:36:11--  https://cocl.us/Geospatial_data
Connecting to cocl.us|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-10 14:36:13--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com... 107.152.25.197, 107.152.24.197
Connecting to ibm.box.com|107.152.25.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-10 14:36:13--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr

In [40]:
geo_df = pd.read_csv('Geospatial_data')

In [41]:
geo_df.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [43]:
toronto_neighborhoods = pd.merge(geo_df, df, right_on='Postal code', left_on='Postal Code', how='right')

In [45]:
toronto_neighborhoods = toronto_neighborhoods[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

In [46]:
toronto_neighborhoods

Postal Code      Borough  \
0           M1B  Scarborough   
1           M1C  Scarborough   
2           M1E  Scarborough   
3           M1G  Scarborough   
4           M1H  Scarborough   
..          ...          ...   
98          M9N         York   
99          M9P    Etobicoke   
100         M9R    Etobicoke   
101         M9V    Etobicoke   
102         M9W    Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                      Malvern / Rouge  43.806686 -79.194353  
1             Rouge Hill / Port Union / Highland Creek  43.784535 -79.160497  
2                  Guildwood / Morningside / West Hill  43.763573 -79.188711  
3                                               Woburn  43.770992 -79.216917  
4                                            Cedarbrae  43.773136 -79.239476  
..                                                 ...        ...        ...  
98                                              Weston  43.706876 -79.518188  
99                                           Westmount  43.696319 -79.532242  
100  Kingsview Village / St. Phillips / Martin Grov...  43.688905 -79.554724  
101  South Steeles / Silverstone / Humbergate / Jam...  43.739416 -79.588437  
102                                          Northwest  43.706748 -79.594054  

[103 rows x 5 columns]

## PART THREE GEOLOCATION AND CLUSTERING

In [65]:
import geocoder 
import folium

In [56]:
toronto_nb = toronto_neighborhoods[toronto_neighborhoods['Borough'].str.contains('Toronto')]

In [57]:
toronto_nb.shape

(39, 5)

In [63]:
toronto_nb.head()

Postal Code          Borough                     Neighborhood   Latitude  \
37         M4E     East Toronto                      The Beaches  43.676357   
41         M4K     East Toronto    The Danforth West / Riverdale  43.679557   
42         M4L     East Toronto  India Bazaar / The Beaches West  43.668999   
43         M4M     East Toronto                  Studio District  43.659526   
44         M4N  Central Toronto                    Lawrence Park  43.728020   

    Longitude  
37 -79.293031  
41 -79.352188  
42 -79.315572  
43 -79.340923  
44 -79.388790

In [62]:
# FIND GEOLOCATION INFO FOR TORONTO


# initialize your variable to None
lat_lng_coords = None


g = geocoder.google('Toronto, Ontario')
g

<[REQUEST_DENIED] Google - Geocode [empty]>

In [68]:
t_lat = 43.6532
t_long = -79.3832

In [71]:
# Map of Toronto
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[t_lat, t_long], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_nb['Latitude'], toronto_nb['Longitude'], toronto_nb['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# PRELIMINARY THOUGHTS:
CLUSTERING Groups will probably revolve around BUSINESS, RESIDENTIAL, TOURIST SPOTS, NIGHTLIFE, WEEKEND SPOTS

In [76]:
import requests
# FOURSQUARE CREDS
CLIENT_ID = '0I4KDZAGDIBSJSWLA0KIG35A3DCPGHYXETFHRHGURJ31HDFW' # your Foursquare ID
CLIENT_SECRET = 'ITRLXIQPIFEDIJZVSKYB3YGHFNFOXUWOIEU1CHSQGDXVMGIE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0I4KDZAGDIBSJSWLA0KIG35A3DCPGHYXETFHRHGURJ31HDFW
CLIENT_SECRET:ITRLXIQPIFEDIJZVSKYB3YGHFNFOXUWOIEU1CHSQGDXVMGIE


## BELOW IS CODE FROM THE NEW YORK CLUSTERING NOTEBOOK

In [77]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [78]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [79]:
# Run FUNCTION

toronto_venues = getNearbyVenues(names=toronto_nb['Neighborhood'],
                                   latitudes=toronto_nb['Latitude'],
                                   longitudes=toronto_nb['Longitude']
                                  )



The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park /

## IS THIS A REAL NEIGHBORHOOD!?

In [84]:
toronto_nb[toronto_nb['Neighborhood']=='Business reply mail Processing CentrE']

Postal Code       Borough                           Neighborhood  \
87         M7Y  East Toronto  Business reply mail Processing CentrE   

     Latitude  Longitude  
87  43.662744 -79.321558

In [85]:
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Berczy Park                                                                                                          30
Brockton / Parkdale Village / Exhibition Place                                                                       30
Business reply mail Processing CentrE                                                                                30
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport    30
Central Bay Street                                                                                                   30
Christie                                                                                                             30
Church and Wellesley                                                                                                 30
Commerce Court / Victoria Hotel                                                                                      30
Davisville                 

In [86]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 200 uniques categories.


In [87]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Airport  Airport Lounge  American Restaurant  Amphitheater  \
0            0        0               0                    0             0   
1            0        0               0                    0             0   
2            0        0               0                    0             0   
3            0        0               0                    0             0   
4            0        0               0                    0             0   

   Aquarium  Art Gallery  Arts & Crafts Store  Asian Restaurant  \
0         0            0                    0                 0   
1         0            0                    0                 0   
2         0            0                    0                 0   
3         0            0                    0                 0   
4         0            0                    0                 0   

   Athletics & Sports  ...  Tibetan Restaurant  Toy / Game Store  Track  \
0                   0  ...                   0                 0      0   
1                   0  ...                   0                 0      0   
2                   0  ...                   0                 0      0   
3                   0  ...                   0                 0      0   
4                   0  ...                   0                 0      0   

   Trail  Train Station  Turkish Restaurant  Vegetarian / Vegan Restaurant  \
0      0              0                   0                              0   
1      0              0                   0                              0   
2      0              0                   0                              1   
3      0              0                   0                              0   
4      0              0                   0                              0   

   Vietnamese Restaurant  Wine Bar  Wings Joint  
0                      0         0            0  
1                      0         0            0  
2                      0         0            0  
3                      0         0            0  
4                      0         0            0  

[5 rows x 200 columns]

In [88]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio   Airport  \
0                                         Berczy Park     0.000000  0.000000   
1      Brockton / Parkdale Village / Exhibition Place     0.000000  0.000000   
2               Business reply mail Processing CentrE     0.000000  0.000000   
3   CN Tower / King and Spadina / Railway Lands / ...     0.033333  0.033333   
4                                  Central Bay Street     0.033333  0.000000   
5                                            Christie     0.000000  0.000000   
6                                Church and Wellesley     0.000000  0.000000   
7                     Commerce Court / Victoria Hotel     0.000000  0.000000   
8                                          Davisville     0.033333  0.000000   
9                                    Davisville North     0.033333  0.000000   
10                      Dufferin / Dovercourt Village     0.000000  0.000000   
11            First Canadian Place / Underground city     0.000000  0.000000   
12                           Forest Hill North & West     0.033333  0.000000   
13                           Garden District, Ryerson     0.000000  0.000000   
14  Harbourfront East / Union Station / Toronto Is...     0.000000  0.000000   
15                     High Park / The Junction South     0.033333  0.000000   
16                    India Bazaar / The Beaches West     0.000000  0.000000   
17        Kensington Market / Chinatown / Grange Park     0.000000  0.000000   
18                                      Lawrence Park     0.000000  0.000000   
19                          Little Portugal / Trinity     0.033333  0.000000   
20                       Moore Park / Summerhill East     0.000000  0.000000   
21                                 North Toronto West     0.066667  0.000000   
22                            Parkdale / Roncesvalles     0.000000  0.000000   
23       Queen's Park / Ontario Provincial Government     0.033333  0.000000   
24                         Regent Park / Harbourfront     0.000000  0.000000   
25                         Richmond / Adelaide / King     0.000000  0.000000   
26                                           Rosedale     0.033333  0.000000   
27                                           Roselawn     0.033333  0.000000   
28                                Runnymede / Swansea     0.000000  0.000000   
29                                     St. James Town     0.000000  0.000000   
30                       St. James Town / Cabbagetown     0.000000  0.000000   
31                                     Stn A PO Boxes     0.000000  0.000000   
32                                    Studio District     0.000000  0.000000   
33  Summerhill West / Rathnelly / South Hill / For...     0.000000  0.000000   
34              The Annex / North Midtown / Yorkville     0.000000  0.000000   
35                                        The Beaches     0.000000  0.000000   
36                      The Danforth West / Riverdale     0.033333  0.000000   
37          Toronto Dominion Centre / Design Exchange     0.000000  0.000000   
38                    University of Toronto / Harbord     0.033333  0.000000   

    Airport Lounge  American Restaurant  Amphitheater  Aquarium  Art Gallery  \
0         0.000000             0.000000      0.000000  0.000000     0.033333   
1         0.000000             0.033333      0.000000  0.000000     0.000000   
2         0.000000             0.000000      0.000000  0.000000     0.000000   
3         0.033333             0.000000      0.000000  0.000000     0.000000   
4         0.000000             0.000000      0.000000  0.000000     0.000000   
5         0.000000             0.000000      0.000000  0.000000     0.000000   
6         0.000000             0.000000      0.000000  0.000000     0.000000   
7         0.000000             0.033333      0.000000  0.000000     0.033333   
8         0.000000             0.000000      0.000000  0.000000     0.000000   
9         0.000000             0.000000 

In [89]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [91]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park    Seafood Restaurant   
1     Brockton / Parkdale Village / Exhibition Place                Bakery   
2              Business reply mail Processing CentrE               Brewery   
3  CN Tower / King and Spadina / Railway Lands / ...                  Park   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0        Farmers Market          Cocktail Bar           Coffee Shop   
1           Coffee Shop            Restaurant                  Café   
2           Pizza Place                  Park                 Beach   
3       Harbor / Marina           Coffee Shop                   Gym   
4                  Café   Japanese Restaurant              Tea Room   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                Beer Bar                Bakery          Concert Hall   
1  Furniture / Home Store             Gift Shop                 Hotel   
2             Coffee Shop        Farmers Market         Burrito Place   
3             Event Space      Sculpture Garden        Scenic Lookout   
4         Bubble Tea Shop        Sandwich Place    Chinese Restaurant   

     8th Most Common Venue  9th Most Common Venue 10th Most Common Venue  
0                 Fountain                   Café                 Lounge  
1  Comfort Food Restaurant     Mexican Restaurant           Cocktail Bar  
2              Snack Place   Pakistani Restaurant            Flea Market  
3         Ramen Restaurant  Performing Arts Venue           Nudist Beach  
4       Italian Restaurant                    Spa             Steakhouse

In [95]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 1, 3, 0, 0, 3, 2, 2], dtype=int32)

In [100]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_nb

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_nb.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Postal Code          Borough                     Neighborhood   Latitude  \
37         M4E     East Toronto                      The Beaches  43.676357   
41         M4K     East Toronto    The Danforth West / Riverdale  43.679557   
42         M4L     East Toronto  India Bazaar / The Beaches West  43.668999   
43         M4M     East Toronto                  Studio District  43.659526   
44         M4N  Central Toronto                    Lawrence Park  43.728020   

    Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
37 -79.293031               0                 Beach             BBQ Joint   
41 -79.352188               0      Greek Restaurant                  Park   
42 -79.315572               1                  Park               Brewery   
43 -79.340923               0                   Bar           Pizza Place   
44 -79.388790               0                Bakery                  Park   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
37                   Pub           Coffee Shop             Gastropub   
41        Ice Cream Shop    Italian Restaurant                Bakery   
42                 Beach                  Café           Pizza Place   
43    Italian Restaurant           Coffee Shop        Ice Cream Shop   
44                  Café           Coffee Shop      Sushi Restaurant   

      6th Most Common Venue 7th Most Common Venue      8th Most Common Venue  \
37                     Café                  Park  Middle Eastern Restaurant   
41  Fruit & Vegetable Store         Grocery Store                  Juice Bar   
42        Indian Restaurant           Coffee Shop              Burrito Place   
43                Bookstore              Boutique                 Restaurant   
44                 Tea Room        Sandwich Place               Burger Joint   

            9th Most Common Venue 10th Most Common Venue  
37  Vegetarian / Vegan Restaurant    Japanese Restaurant  
41                            Spa         Cosmetics Shop  
42                    Snack Place           Soccer Field  
43                    Fish Market             Taco Place  
44                Bubble Tea Shop            Flower Shop

In [103]:
# create map
map_clusters = folium.Map(location=[t_lat, t_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [104]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



Borough  Cluster Labels          1st Most Common Venue  \
37      East Toronto               0                          Beach   
41      East Toronto               0               Greek Restaurant   
43      East Toronto               0                            Bar   
44   Central Toronto               0                         Bakery   
48   Central Toronto               0                           Park   
49   Central Toronto               0               Sushi Restaurant   
50  Downtown Toronto               0                           Park   
51  Downtown Toronto               0                           Park   
52  Downtown Toronto               0                        Gay Bar   
55  Downtown Toronto               0            Japanese Restaurant   
56  Downtown Toronto               0             Seafood Restaurant   
58  Downtown Toronto               0                           Café   
59  Downtown Toronto               0                          Hotel   
65   Central Toronto               0  Vegetarian / Vegan Restaurant   
66  Downtown Toronto               0                         Bakery   
67  Downtown Toronto               0                           Café   
69  Downtown Toronto               0                         Bakery   
75  Downtown Toronto               0                           Café   
77      West Toronto               0                            Bar   
78      West Toronto               0                         Bakery   
82      West Toronto               0                           Café   
83      West Toronto               0                            Bar   

    2nd Most Common Venue        3rd Most Common Venue  \
37              BBQ Joint                          Pub   
41                   Park               Ice Cream Shop   
43            Pizza Place           Italian Restaurant   
44                   Park                         Café   
48     Italian Restaurant                          Spa   
49                    Spa           Italian Restaurant   
50                    Spa                  Coffee Shop   
51                   Café                    Gastropub   
52            Coffee Shop                     Creperie   
55            Coffee Shop                    Gastropub   
56         Farmers Market                 Cocktail Bar   
58            Pizza Place             Asian Restaurant   
59                  Plaza                         Park   
65     Italian Restaurant                         Café   
66                   Park                    Bookstore   
67  Vietnamese Restaurant           Mexican Restaurant   
69           Cocktail Bar                     Beer Bar   
75           Cocktail Bar                  Coffee Shop   
77       Asian Restaurant        Vietnamese Restaurant   
78            Coffee Shop                   Restaurant   
82                    Bar                  Coffee Shop   
83             Restaurant  Eastern European Restaurant   

            4th Most Common Venue   5th Most Common Venue  \
37                    Coffee Shop               Gastropub   
41             Italian Restaurant                  Bakery   
43                    Coffee Shop          Ice Cream Shop   
44                    Coffee Shop        Sushi Restaurant   
48               Sushi Restaurant                Tea Room   
49                   Liquor Store                    Café   
50                    Yoga Studio          Farmers Market   
51            Japanese Restaurant                   Diner   
52                Bubble Tea Shop            Burger Joint   
55                          Hotel                    Café   
56                    Coffee Shop                Beer Bar   
58                    Coffee Shop              Smoke Shop   
59                         Lounge             Supermarket   
65                  Grocery Store             Pizza Place   
66  Vegetarian / Vegan Restaurant                    Café   
67                   Burger Joint             Coffee Shop   
69     

In [105]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



Borough  Cluster Labels 1st Most Common Venue  \
42      East Toronto               1                  Park   
68  Downtown Toronto               1                  Park   
87      East Toronto               1               Brewery   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
42               Brewery                 Beach                  Café   
68       Harbor / Marina           Coffee Shop                   Gym   
87           Pizza Place                  Park                 Beach   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
42           Pizza Place     Indian Restaurant           Coffee Shop   
68           Event Space      Sculpture Garden        Scenic Lookout   
87           Coffee Shop        Farmers Market         Burrito Place   

   8th Most Common Venue  9th Most Common Venue 10th Most Common Venue  
42         Burrito Place            Snack Place           Soccer Field  
68      Ramen Restaurant  Performing Arts Venue           Nudist Beach  
87           Snack Place   Pakistani Restaurant            Flea Market

In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



Borough  Cluster Labels 1st Most Common Venue  \
45  Central Toronto               2    Italian Restaurant   
46  Central Toronto               2    Italian Restaurant   
47  Central Toronto               2    Italian Restaurant   
63  Central Toronto               2    Italian Restaurant   
64  Central Toronto               2    Italian Restaurant   
76     West Toronto               2                  Café   
84     West Toronto               2                  Café   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
45                  Café           Coffee Shop             Bookstore   
46           Yoga Studio                  Café             Bookstore   
47           Coffee Shop          Dessert Shop                  Café   
63           Coffee Shop                  Café                Bakery   
64                  Café                 Trail           Coffee Shop   
76    Italian Restaurant           Coffee Shop                   Bar   
84    Italian Restaurant           Pizza Place           Coffee Shop   

   5th Most Common Venue      6th Most Common Venue  \
45           Pizza Place              Deli / Bodega   
46           Coffee Shop                       Park   
47             Bookstore                Yoga Studio   
63             Bookstore                Yoga Studio   
64                  Park  Middle Eastern Restaurant   
76                Bakery      Portuguese Restaurant   
84            Taco Place                     Bakery   

        7th Most Common Venue 8th Most Common Venue  9th Most Common Venue  \
45                 Restaurant           Flower Shop      Food & Drink Shop   
46                        Gym                Bakery     Chinese Restaurant   
47                Supermarket     Indian Restaurant  Indonesian Restaurant   
63               Gourmet Shop                 Plaza             Restaurant   
64                Yoga Studio                Bakery              Bookstore   
76  Middle Eastern Restaurant      Sushi Restaurant               Beer Bar   
84                 Shoe Store         Burrito Place      Fish & Chips Shop   

       10th Most Common Venue  
45                Supermarket  
46      Vietnamese Restaurant  
47  Middle Eastern Restaurant  
63               Burger Joint  
64      Indonesian Restaurant  
76       Brazilian Restaurant  
84                    Dog Run

In [107]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



Borough  Cluster Labels 1st Most Common Venue  \
53  Downtown Toronto               3           Coffee Shop   
54  Downtown Toronto               3           Coffee Shop   
57  Downtown Toronto               3           Coffee Shop   
60  Downtown Toronto               3           Coffee Shop   
61  Downtown Toronto               3                  Café   
70  Downtown Toronto               3           Coffee Shop   
85  Downtown Toronto               3           Coffee Shop   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
53                  Park                Bakery        Breakfast Spot   
54                  Café               Theater        Clothing Store   
57                  Café   Japanese Restaurant              Tea Room   
60            Restaurant   Japanese Restaurant         Deli / Bodega   
61            Restaurant           Coffee Shop         Deli / Bodega   
70                  Café             Gastropub                 Hotel   
85                  Park             Gastropub           Wings Joint   

   5th Most Common Venue 6th Most Common Venue      7th Most Common Venue  \
53    Mexican Restaurant        Farmers Market   Mediterranean Restaurant   
54                 Plaza        Sandwich Place              Burrito Place   
57       Bubble Tea Shop        Sandwich Place         Chinese Restaurant   
60                  Café              Beer Bar                      Hotel   
61   Japanese Restaurant             Gastropub                     Bakery   
70            Restaurant    Seafood Restaurant                Pizza Place   
85                 Diner           Pizza Place  Middle Eastern Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
53                   Spa        Chocolate Shop     Italian Restaurant  
54          Burger Joint         Shopping Mall    Japanese Restaurant  
57    Italian Restaurant                   Spa             Steakhouse  
60                   Pub                Museum                    Bar  
61            Steakhouse        Ice Cream Shop               Beer Bar  
70                   Bar      Greek Restaurant                    Gym  
85    Mexican Restaurant            Hobby Shop    Distribution Center

### FINAL THOUGHTS

I feel like cluster 0 might point to residental areas (lots of dog parks) or . 
Cluster 1 seems touristy- or 'things to do on the weekend'
Cluster 2 has a lot of yoga studios
Cluster 3 has a lot of bars and hotels and restaurants--> more active nightlight?